In [46]:
import math
import numpy as np
import pandas as pd

In [99]:
# Load dataset 
raw_train = pd.read_csv('data/train.csv')
raw_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [100]:
## Missing data imputation (Age, Embarked)

# Impute age using the average age of one's given pronounciation 
# Mr., Miss., Master., Mrs., and else (including Don., Rev., Dr., Mlle., Col., Capt.)
clean_train = raw_train
index_Mr = list()
index_Miss = list()
index_Master = list()
index_Mrs = list()
index_else = list()
for name in clean_train['Name'].iteritems():
    if 'Mr.' in name[1]:
        index_Mr.append(name[0])
    elif 'Miss.' in name[1]:
        index_Miss.append(name[0])
    elif 'Master.' in name[1]:
        index_Master.append(name[0])
    elif 'Mrs.' in name[1]:
        index_Mrs.append(name[0])
    else:
        index_else.append(name[0]) 

Prefix_Mr = clean_train.loc[index_Mr]
Prefix_Miss = clean_train.loc[index_Miss]
Prefix_Master = clean_train.loc[index_Master]
Prefix_Mrs = clean_train.loc[index_Mrs]
Prefix_else = clean_train.loc[index_else]

Prefix_Mr['Age'] = Prefix_Mr['Age'].map(lambda x: Prefix_Mr['Age'].mean() if math.isnan(x) else x)
Prefix_Miss['Age'] = Prefix_Miss['Age'].map(lambda x: Prefix_Miss['Age'].mean() if math.isnan(x) else x)
Prefix_Master['Age'] = Prefix_Master['Age'].map(lambda x: Prefix_Master['Age'].mean() if math.isnan(x) else x)
Prefix_Mrs['Age'] = Prefix_Mrs['Age'].map(lambda x: Prefix_Mrs['Age'].mean() if math.isnan(x) else x)
Prefix_else['Age'] = Prefix_else['Age'].map(lambda x: Prefix_else['Age'].mean() if math.isnan(x) else x)

clean_train.loc[index_Mr] = Prefix_Mr
clean_train.loc[index_Miss] = Prefix_Miss
clean_train.loc[index_Master] = Prefix_Master
clean_train.loc[index_Mrs] = Prefix_Mrs
clean_train.loc[index_else] = Prefix_else

# Impute Embarked using the most frequent type of Embarked symbol
clean_train[clean_train['Embarked'].isna()].loc[slice(None), 'Embarked'] = clean_train['Embarked'].mode().iloc[0]

In [101]:
print(clean_train)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex        Age  \
0                              Braund, Mr. Owen Harris    male  22.000000   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.000000   
2                               Heikkinen, Miss. Laina  female  26.000000   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.000000   
4                             Allen, Mr. William Henry    male  35.000000   
..                                                 ...     ...        ...   
886 

In [102]:
clean_train.to_csv('data/clean_train.csv') 